In [1]:
!pip install split-folders

In [11]:
import sys
sys.path.append('/kaggle/input/major-project-code/main_folder/')

In [3]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import keras
import splitfolders
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from keras.layers import Input
from keras.models import Model
from code_base.model.model import HARModel
from code_base.utils.lr_schedule import lr_schedule
# from code_base.utils.constants import img_cols, img_rows
from code_base.data_pps.datagenffd import PreprocessData

In [4]:
COLS = 224
ROWS = 224
CHANS = 3
NUM_CLASSES = 15
DIR = '/kaggle/input/human-activity-recognition-train-and-test-folders/HAR/train'
# VALIDATION_DIR = ''

In [5]:
splitfolders.ratio(DIR, 
                   output='/kaggle/working/', 
                   seed=1337, 
                   ratio=(0.75,0.25)
                  )

Copying files: 12600 files [00:56, 223.48 files/s]


In [6]:
TRAINING_DIR = '/kaggle/working/train/'
VALIDATION_DIR = '/kaggle/working/val/'
for _ in tqdm(range(1)):
    train_generator = PreprocessData.transform(
                        DIR = TRAINING_DIR,
                        batch_size = 16,
                        shuffle = True,
                        )
    validation_generator = PreprocessData.transform(
                        DIR = VALIDATION_DIR,
                        batch_size = 16,
                        shuffle = False,
                        )

  0%|          | 0/1 [00:00<?, ?it/s]

Found 9450 images belonging to 15 classes.
Found 3150 images belonging to 15 classes.


100%|██████████| 1/1 [00:00<00:00,  4.04it/s]


In [7]:
inp = Input((ROWS, COLS, CHANS))
x = HARModel(num_classes= NUM_CLASSES)(inp)
model = Model(inputs=inp, outputs=x)
# model.summary(expand_nested=True)

In [8]:
#Training

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(),
              metrics=["accuracy"])
lr_callback = LearningRateScheduler(lr_schedule)
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs=25,
                    batch_size=16, 
                    verbose=1,
                    callbacks=[ lr_callback]
                   )

Epoch 1/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 104s 130ms/step - accuracy: 0.0882 - loss: 4.1127 - val_accuracy: 0.0924 - val_loss: 2.7545 - learning_rate: 0.0100
Epoch 2/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 59s 100ms/step - accuracy: 0.1115 - loss: 2.6576 - val_accuracy: 0.1168 - val_loss: 2.6204 - learning_rate: 0.0100
Epoch 3/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 59s 99ms/step - accuracy: 0.1395 - loss: 2.5674 - val_accuracy: 0.1333 - val_loss: 2.9606 - learning_rate: 0.0100
Epoch 4/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - accuracy: 0.1760 - loss: 2.4622 - val_accuracy: 0.0924 - val_loss: 2.8228 - learning_rate: 0.0100
Epoch 5/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 60s 100ms/step - accuracy: 0.1909 - loss: 2.4167 - val_accuracy: 0.1933 - val_loss: 2.5822 - learning_rate: 0.0100
Epoch 6/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 59s 99ms/step - accuracy: 0.2068 - loss: 2.3595 - val_accuracy: 0.1521 - val_loss: 2.5260 - learning_rate: 0.0100
Epoch 7/25
591/591 ━━━━━━━━━━━━━━━━━━━━ 59s 99ms/step - accuracy: 0.224

In [9]:
model.save('/kaggle/working/model_DIAT_HAR_vanilla.keras')